In [1]:
%reload_ext lab_black

In [2]:
import numpy as np
from scipy.optimize import minimize_scalar
from worstcase import param, derive, unit

In [3]:
def ebers_moll_model(Is, Vbe, T):
    q = 1.602176e-19 * unit.C
    k = 1.3806503e-23 * unit("J/K")
    return Is * (np.exp(q * Vbe / (k * T)) - 1)

In [4]:
def dmmt3906(Vbe, T):
    return ebers_moll_model(20.3 * unit.fA, Vbe, T).to(unit.mA)

In [5]:
def dmmt3906_unitless(Vbe, T):
    return dmmt3906(Vbe * unit.V, T * unit.K).to(unit.A).m

In [6]:
VBE_MISMATCH = param.bytol(nom=0 * unit.mV, tol=2 * unit.mV, rel=False)
HFE_MISMATCH = param.bytol(nom=0, tol=0.02, rel=False)
HFE = param.byrange(nom=150, lb=80, ub=300)
R1 = param.bytol(nom=8.2 * unit.kohm, tol=0.01, rel=True)
R2 = param.bytol(nom=4.7 * unit.kohm, tol=0.01, rel=True)

In [7]:
@derive.byev(
    r1=R1, r2=R2, hfe1=HFE, vbe_mismatch=VBE_MISMATCH, hfe_mismatch=HFE_MISMATCH
)
def IDIODE(vout, vgsth, rdson, r1, r2, hfe1, vbe_mismatch, hfe_mismatch, temp):
    # the system of equations will be solved numerically
    def idiode_error(idiode_estimate):
        def inverse_dmmt3906(ic):
            vbefun = lambda vbe: dmmt3906_unitless(vbe, temp.to(unit.K).m)
            minfun = lambda vbe: (vbefun(vbe) - ic.to(unit.A).m) ** 2
            result = minimize_scalar(minfun)
            assert result.success
            return result.x * unit.V

        vc2 = vout - vgsth  # pass transistor is "barely on" (at threshold)
        ic2 = vc2 / r2
        vbe2 = inverse_dmmt3906(ic2)
        hfe2 = hfe1 * (1 + hfe_mismatch)
        ib2 = ic2 / hfe2
        vb = vout - vbe2

        ie1 = vb / r1 - ib2
        ic1 = ie1 * hfe1 / (1 + hfe1)
        ib1 = ic1 / hfe1
        vbe1 = inverse_dmmt3906(ic1)
        vbe1 += vbe_mismatch
        vin = vb + vbe1

        # print("Q2:", vout, vb, vc2, ic2.to(unit.mA), ib2.to(unit.mA))
        # print("Q1:", vin, ic1.to(unit.mA), ib1.to(unit.mA))

        idiode_actual = (vin - vout) / rdson
        return (idiode_actual.to(unit.A).m - idiode_estimate) ** 2

    result = minimize_scalar(idiode_error)
    assert result.success
    return result.x * unit.A

In [10]:
IDIODE(
    vout=3.3 * unit.V,
    vgsth=param.byrange(nom=1.0 * unit.V, lb=0.5 * unit.V, ub=1.5 * unit.V),
    rdson=param.byrange(nom=0.6 * unit.ohm, lb=0.4 * unit.ohm, ub=0.9 * unit.ohm),
    temp=300 * unit.K,
)  # note: SI2301CDS mosfet with 390mohm series resistance

-18.08 mA (nom), -47.53 mA (lb), -1.869 mA (ub)

In [11]:
0.9 * 0.125  # max forward voltage drop when USB unpowered

0.1125